<strong>Data Source:</strong><br>
  <strong>Student Depression Dataset, Shodolamu Opeyemi.</strong><br>
  <strong>Retrieved from</strong>
  <a href="https://www.kaggle.com/datasets/hopesb/student-depression-dataset" target="_blank">
    https://www.kaggle.com/datasets/hopesb/student-depression-dataset
  </a>


<p>
  In this analysis, I used a logistic regression model to predict the likelihood of depression among students based on various academic, lifestyle, and personal indicators. The dataset included features such as age, academic and work pressure, CGPA, study satisfaction, sleep duration, dietary habits, financial stress, and family mental health history. I performed data cleaning, handled categorical variables through one-hot encoding, and addressed multicollinearity using the Variance Inflation Factor (VIF). The model was trained on 80% of the data and evaluated on the remaining 20% using statsmodels in Python. The final model provided interpretable coefficients and demonstrated how certain lifestyle and academic factors may be statistically associated with student depression.
</p>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

<p>
  The dataset was pre-split into training and testing subsets before analysis. I used the <code>train_data_80.csv</code> file, which contains 80% of the original data to train the logistic regression model. The remaining 20% of the data was stored separately in <code>test_data_20.csv</code> and used later to evaluate the model's performance on unseen data. This separation helps prevent data leakage and ensures a fair assessment of the model’s generalization ability.
</p>

In [2]:
raw_data = pd.read_csv('train_data_80.csv') #the data already split in advanced
raw_data

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,66085,Male,28.0,Varanasi,Student,2.0,0.0,8.29,5.0,0.0,'5-6 hours',Moderate,MBA,No,4.0,1.0,Yes,0
1,123583,Female,33.0,Patna,Student,4.0,0.0,9.05,4.0,0.0,'Less than 5 hours',Healthy,M.Com,No,12.0,4.0,Yes,0
2,77220,Female,33.0,Jaipur,Student,4.0,0.0,8.08,4.0,0.0,'Less than 5 hours',Healthy,MA,Yes,12.0,3.0,No,1
3,113182,Female,29.0,Kanpur,Student,2.0,0.0,5.76,4.0,0.0,'5-6 hours',Moderate,M.Ed,No,10.0,2.0,Yes,0
4,94866,Female,20.0,Surat,Student,5.0,0.0,5.77,5.0,0.0,'7-8 hours',Moderate,'Class 12',Yes,11.0,5.0,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,108952,Female,34.0,Ahmedabad,Student,2.0,0.0,7.79,4.0,0.0,'5-6 hours',Unhealthy,B.Arch,No,10.0,1.0,Yes,1
22316,27204,Male,20.0,Mumbai,Student,3.0,0.0,5.82,1.0,0.0,'More than 8 hours',Moderate,'Class 12',No,3.0,4.0,No,0
22317,4417,Male,18.0,Kolkata,Student,5.0,0.0,6.37,3.0,0.0,'Less than 5 hours',Moderate,'Class 12',Yes,6.0,5.0,Yes,1
22318,79872,Female,18.0,Chennai,Student,3.0,0.0,7.21,4.0,0.0,'More than 8 hours',Unhealthy,'Class 12',Yes,0.0,3.0,Yes,0


Checking for null or missing value

In [3]:
raw_data.isnull().sum()

id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64

In [4]:
raw_data.describe(include='all')

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
count,22320.000000,22320,22320.000000,22320,22320,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320,22320,22320,22320,22320.000000,22320,22320,22320.000000
unique,NaN,2,NaN,47,12,NaN,NaN,NaN,NaN,NaN,5,4,28,2,NaN,6,2,NaN
top,NaN,Male,NaN,Kalyan,Student,NaN,NaN,NaN,NaN,NaN,'Less than 5 hours',Unhealthy,'Class 12',Yes,NaN,5.0,No,NaN
freq,NaN,12462,NaN,1255,22295,NaN,NaN,NaN,NaN,NaN,6629,8235,4874,14164,NaN,5362,11497,NaN
mean,70471.890233,NaN,25.812455,NaN,NaN,3.147312,0.000538,7.647659,2.941353,0.000851,NaN,NaN,NaN,NaN,7.146729,NaN,NaN,0.586828
std,40689.984378,NaN,4.901516,NaN,NaN,1.380119,0.049185,1.472603,1.361268,0.049634,NaN,NaN,NaN,NaN,3.712050,NaN,NaN,0.492414
min,2.000000,NaN,18.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
25%,35011.500000,NaN,21.000000,NaN,NaN,2.000000,0.000000,6.270000,2.000000,0.000000,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,0.000000
50%,70808.500000,NaN,25.000000,NaN,NaN,3.000000,0.000000,7.770000,3.000000,0.000000,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,1.000000
75%,105839.250000,NaN,30.000000,NaN,NaN,4.000000,0.000000,8.920000,4.000000,0.000000,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,1.000000


<p>
This dataset contains 22,320 records of students with the goal of predicting whether a student is likely to experience depression based on various academic, lifestyle, and demographic factors. The average age of students is around 26 years, ranging from 18 to 59, suggesting a diverse range of education levels and life stages. Academic pressure, CGPA, and study satisfaction are included as indicators of academic performance and stress, while lifestyle-related features such as sleep duration, dietary habits, and work/study hours provide insight into personal well-being.
</p>

In [5]:
data = raw_data.drop(['id','City'], axis=1)

In [6]:
data = pd.get_dummies(
    data,
    columns=['Gender','Profession','Sleep Duration','Dietary Habits','Degree','Have you ever had suicidal thoughts ?','Family History of Mental Illness','Financial Stress'],
    prefix=['Gender','Profession','Sleep Duration','Dietary Habits','Degree','Have you ever had suicidal thoughts ?','Family History of Mental Illness','Financial Stress'],
    drop_first=True, dtype=int  # optional: drops one column per group to avoid dummy variable trap
)
data

,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Depression,Gender_Male,Profession_'Content Writer',...,Degree_MSc,Degree_Others,Degree_PhD,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes,Financial Stress_2.0,Financial Stress_3.0,Financial Stress_4.0,Financial Stress_5.0,Financial Stress_?
0,28.0,2.0,0.0,8.29,5.0,0.0,4.0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,33.0,4.0,0.0,9.05,4.0,0.0,12.0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,33.0,4.0,0.0,8.08,4.0,0.0,12.0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
3,29.0,2.0,0.0,5.76,4.0,0.0,10.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,20.0,5.0,0.0,5.77,5.0,0.0,11.0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,34.0,2.0,0.0,7.79,4.0,0.0,10.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
22316,20.0,3.0,0.0,5.82,1.0,0.0,3.0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
22317,18.0,5.0,0.0,6.37,3.0,0.0,6.0,1,1,0,...,0,0,0,1,1,0,0,0,1,0
22318,18.0,3.0,0.0,7.21,4.0,0.0,0.0,0,0,0,...,0,0,0,1,1,0,1,0,0,0


<p>
To prepare the dataset for logistic regression, I first dropped the <code>id</code> and <code>city</code> columns. The <code>id</code> column serves only as a unique identifier for each row and contains no useful information for prediction. Including it would add noise to the model without contributing any explanatory power. The <code>city</code> column was removed due to its high cardinality — it contains a large number of unique values that would significantly increase the dimensionality of the data when one-hot encoded, potentially introducing sparsity and multicollinearity without improving predictive performance.
</p>

<p>
Next, I applied <strong>one-hot encoding</strong> to convert categorical variables such as <code>Gender</code>, <code>Profession</code>, <code>Sleep Duration</code>, <code>Dietary Habits</code>, and <code>Degree</code> into a numerical format suitable for logistic regression. This method creates binary columns for each category, allowing the model to learn from categorical information without assuming any ordinal relationship. To avoid the dummy variable trap and multicollinearity, one category from each group was dropped using the <code>drop_first=True</code> parameter.
</p>

In [7]:
data.describe(include='all')

,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Depression,Gender_Male,Profession_'Content Writer',...,Degree_MSc,Degree_Others,Degree_PhD,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes,Financial Stress_2.0,Financial Stress_3.0,Financial Stress_4.0,Financial Stress_5.0,Financial Stress_?
count,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,...,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000,22320.000000
mean,25.812455,3.147312,0.000538,7.647659,2.941353,0.000851,7.146729,0.586828,0.558333,0.000045,...,0.042115,0.001299,0.018548,0.634588,0.484901,0.181362,0.185977,0.207975,0.240233,0.000134
std,4.901516,1.380119,0.049185,1.472603,1.361268,0.049634,3.712050,0.492414,0.496597,0.006693,...,0.200855,0.036023,0.134926,0.481556,0.499783,0.385326,0.389097,0.405868,0.427234,0.011593
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,2.000000,0.000000,6.270000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,3.000000,0.000000,7.770000,3.000000,0.000000,8.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30.000000,4.000000,0.000000,8.920000,4.000000,0.000000,10.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,59.000000,5.000000,5.000000,10.000000,5.000000,4.000000,12.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Declaring Variable and Feature Scaling

<p>
To build a logistic regression model, I first separated the target variable by assigning the <code>Depression</code> column to <code>y</code>. This is the binary dependent variable indicating whether a student is likely to be experiencing depression (1) or not (0). The remaining features were stored in <code>x1</code>, which includes all the independent variables used for prediction.
</p>

<p>
Since many of the features vary in scale — such as <code>CGPA</code>, <code>Academic Pressure</code>, and <code>Work/Study Hours</code> — I applied <code>StandardScaler</code> from <code>sklearn.preprocessing</code> to standardize the feature values. This transformation centers the data to have a mean of 0 and a standard deviation of 1. Feature scaling is crucial for models like logistic regression that are sensitive to the magnitude of feature values, as it ensures that no single feature dominates the model due to scale differences.
</p>

In [9]:
y = data['Depression']
x1 = data.drop('Depression', axis=1)
from sklearn.preprocessing import StandardScaler
x1_scaled = pd.DataFrame(StandardScaler().fit_transform(x1), columns=x1.columns)


## Check for multicollinear

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(x1.values, i) for i in range(x1.shape[1])]
vif["feature"] = x1.columns
print(vif.sort_values(by="VIF", ascending=False))

          VIF                                    feature
17  93.404753                         Profession_Student
0   50.382048                                        Age
3   28.364266                                       CGPA
1    6.907463                          Academic Pressure
4    5.866397                         Study Satisfaction
6    4.829411                           Work/Study Hours
53   3.131542  Have you ever had suicidal thoughts ?_Yes
5    2.470135                           Job Satisfaction
2    2.467461                              Work Pressure
58   2.445422                       Financial Stress_5.0
25   2.419030                   Dietary Habits_Unhealthy
20   2.365839         Sleep Duration_'Less than 5 hours'
23   2.321412                    Dietary Habits_Moderate
7    2.311713                                Gender_Male
19   2.206781                 Sleep Duration_'7-8 hours'
57   2.203566                       Financial Stress_4.0
56   2.052689                  

<p>
To detect multicollinearity among the independent variables, I calculated the <strong>Variance Inflation Factor (VIF)</strong> for each feature. VIF measures how much the variance of a regression coefficient is inflated due to linear relationships with other features. A high VIF (typically above 10) indicates strong multicollinearity, which can make coefficient estimates unstable and difficult to interpret.
</p>

<p>
The results showed that <code>Profession_Student</code> (VIF = 93.40), <code>Age</code> (VIF = 50.38), and <code>CGPA</code> (VIF = 28.36) had extremely high VIF scores. To improve model stability and avoid redundancy, I dropped these three variables from the dataset. Removing highly collinear features helps the model focus on variables that contribute unique information, leading to more reliable and interpretable results in the logistic regression.
</p>

In [12]:
data = data.drop(['Age','Profession_Student','CGPA'], axis=1)
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
vif["feature"] = data.columns
print(vif.sort_values(by="VIF", ascending=False))

         VIF                                    feature
0   6.914831                          Academic Pressure
5   4.913799                                 Depression
2   4.616578                         Study Satisfaction
4   4.477978                           Work/Study Hours
51  3.887812  Have you ever had suicidal thoughts ?_Yes
3   2.467911                           Job Satisfaction
1   2.466032                              Work Pressure
56  2.424411                       Financial Stress_5.0
23  2.373249                   Dietary Habits_Unhealthy
6   2.211359                                Gender_Male
21  2.193555                    Dietary Habits_Moderate
18  2.183819         Sleep Duration_'Less than 5 hours'
55  2.093976                       Financial Stress_4.0
17  2.043153                 Sleep Duration_'7-8 hours'
54  1.915570                       Financial Stress_3.0
52  1.902605       Family History of Mental Illness_Yes
19  1.838805         Sleep Duration_'More than 8

# Regession

In [14]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()

         Current function value: 0.344022
         Iterations: 35


C:\Users\Bocah\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [15]:
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Depression   No. Observations:                22320
Model:                          Logit   Df Residuals:                    22259
Method:                           MLE   Df Model:                           60
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                  0.4926
Time:                        23:29:03   Log-Likelihood:                -7678.6
converged:                      False   LL-Null:                       -15133.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=============================================================================================================
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                        16.2910   1.31e+05      0.000      1.000   -2.56e+05    2.56e+05
Age                                          -0.1150      0.006    -20.752      0.000      -0.126      -0.104
Academic Pressure                             0.8527      0.017     50.470      0.000       0.820       0.886
Work Pressure                                 0.2001      0.594      0.337      0.736      -0.964       1.364
CGPA                                          0.0637      0.014      4.549      0.000       0.036       0.091
Study Satisfaction                           -0.2453      0.015    -16.016      0.000      -0.275      -0.215
Job Satisfaction                              0.0931      0.493      0.189      0.850      -0.873       1.059
Work/Study Hours                              0.1172      0.006     20.947      0.000       0.106       0.128
Gender_Male                                   0.0065      0.042      0.155      0.877      -0.075       0.088
Profession_'Content Writer'                  -3.0462   1.31e+05  -2.32e-05      1.000   -2.57e+05    2.57e+05
Profession_'Digital Marketer'               -19.6182   1.31e+05     -0.000      1.000   -2.56e+05    2.56e+05
Profession_'UX/UI Designer'                  -1.7244   1.31e+05  -1.32e-05      1.000   -2.57e+05    2.57e+05
Profession_Architect                        -17.3321   1.31e+05     -0.000      1.000   -2.56e+05    2.56e+05
Profession_Doctor                             6.9739   2.13e+05   3.27e-05      1.000   -4.18e+05    4.18e+05
Profession_Entrepreneur                      -2.3085   1.31e+05  -1.76e-05      1.000   -2.57e+05    2.57e+05
Profession_Lawyer                            -4.1894   1.31e+05   -3.2e-05      1.000   -2.57e+05    2.57e+05
Profession_Manager                           -0.8543   1.31e+05  -6.52e-06      1.000   -2.57e+05    2.57e+05
Profession_Pharmacist                        -0.4292   1.31e+05  -3.27e-06      1.000   -2.57e+05    2.57e+05
Profession_Student                          -19.5482   1.31e+05     -0.000      1.000   -2.56e+05    2.56e+05
Profession_Teacher                          -19.1998   1.31e+05     -0.000      1.000   -2.56e+05    2.56e+05
Sleep Duration_'7-8 hours'                   -0.0315      0.059     -0.538      0.591      -0.146       0.083
Sleep Duration_'Less than 5 hours'            0.3222      0.058      5.548      0.000       0.208       0.436
Sleep Duration_'More than 8 hours'           -0.2958      0.061     -4.823      0.000      -0.416      -0.176
Sleep Duration_Others                        -0.3868      0.681     -0.568      0.570      -1.721       0.948
Dietary Habits_Moderate                       0.4744      0.051      9.314      0.000       0.375       0.574
Dietary Habits_Others                         0.8540      0.835      1.022      0.307      -0.783       2.491
Dietary Habits_Unhealthy                      1.075

<h2>Logistic Regression Analysis Summary</h2>

<h3>Pseudo R-squared (0.4926):</h3>
<p>
This means the model improves the likelihood of predicting depression by about 49% compared to a model with no predictors.
For logistic regression, this is considered a relatively strong model.
</p>

<h3>Log-Likelihood (-7678.6) and LLR p-value (0.000):</h3>
<p>
The very low p-value indicates the overall model is statistically significant.
It suggests that at least one of the predictors contributes meaningfully to the prediction of depression.
</p>

<h3>Key Coefficients and Interpretations:</h3>
<ul>
  <li><b>Academic Pressure (coef = 0.85):</b> Increases the odds of depression. Students with more academic stress are significantly more likely to be depressed.</li>
  <li><b>Study Satisfaction (coef = -0.25):</b> Decreases the odds of depression. More satisfied students are less likely to be depressed.</li>
  <li><b>Work/Study Hours (coef = 0.12):</b> More hours working or studying increases depression risk.</li>
  <li><b>Sleep Duration ('Less than 5 hours', coef = 0.32):</b> Short sleep duration significantly increases the likelihood of depression.</li>
  <li><b>Sleep Duration ('More than 8 hours', coef = -0.30):</b> Longer sleep appears protective against depression.</li>
  <li><b>Unhealthy Diet (coef = 1.08):</b> Strongly increases the odds of depression.</li>
  <li><b>Suicidal Thoughts (coef = 2.56):</b> The strongest predictor — students with suicidal thoughts are nearly 13 times more likely to be depressed.</li>
  <li><b>Family History of Mental Illness (coef = 0.29):</b> Positively associated with depression risk.</li>
  <li><b>Financial Stress (coef ranges from 0.39 to 2.25):</b> Higher financial stress significantly raises the probability of depression.</li>
</ul>

<h3>Model Concerns (for future optimization):</h3>
<ul>
  <li>Several profession-related variables have unusually high standard errors and p-values of 1.0, indicating they do not contribute meaningfully.</li>
  <li>The model failed to fully converge, which may be due to multicollinearity or redundant features.</li>
  <li>These convergence issues suggest that the model can benefit from refinement by removing non-informative variables.</li>
</ul>

<h3>Conclusion:</h3>
<p>
The model identifies key academic, psychological, and lifestyle factors associated with student depression.
While it provides useful insights, simplifying the model by removing multicollinear and statistically insignificant features will improve its reliability and generalizability.
</p>


# Accuracy

Predicted Value by the model

In [18]:
results_log.predict()

array([0.01499068, 0.39272911, 0.77187995, ..., 0.99457025, 0.80526964,
       0.05782646])

Actual Value

In [20]:
np.array(data['Depression'])

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [21]:
results_log.pred_table()

array([[ 7319.,  1903.],
       [ 1446., 11652.]])

In [22]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns =['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,7319.0,1903.0
Actual 1,1446.0,11652.0


This table is a confusion matrix, which evaluates the performance of a binary classification model 

True Negative (7319): The model correctly predicted students who are not depressed.

False Positive (1903): The model incorrectly predicted depression for students who are not depressed.

False Negative (1446): The model missed predicting depression for students who are actually depressed.

True Positive (11652): The model correctly predicted students who are depressed.

In [23]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.8499551971326165

An accuracy above 80% is generally considered strong, especially in social science or health-related datasets.
An accuracy score of 0.8499 (or approximately 85%) for training model means that the logistic regression model correctly predicted the depression status (either yes or no) for about 85% of the students in training data.

## Testing the model and assessing its accuracy

Testing on unseen data helps assess how well our model generalizes. A model might memorize the training data perfectly but fail on new data. By testing, you check whether this model is overfitted (too specific) or well-balanced. We'll do the same cleaning process, changing the categorical into numerical, and then predict with the model we already create.

In [25]:
test = pd.read_csv('test_data_20.csv')
test.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,101205,Female,29.0,Kalyan,Student,2.0,0.0,8.53,3.0,0.0,'More than 8 hours',Healthy,B.Arch,No,10.0,5.0,No,0
1,83727,Male,28.0,Srinagar,Student,2.0,0.0,5.57,5.0,0.0,'7-8 hours',Unhealthy,M.Tech,Yes,3.0,1.0,Yes,0
2,38395,Male,34.0,Varanasi,Student,3.0,0.0,5.12,4.0,0.0,'More than 8 hours',Moderate,M.Pharm,Yes,4.0,3.0,Yes,0
3,107434,Male,21.0,Mumbai,Student,5.0,0.0,8.95,2.0,0.0,'Less than 5 hours',Unhealthy,B.Pharm,Yes,8.0,1.0,No,1
4,79662,Male,25.0,Visakhapatnam,Student,5.0,0.0,7.87,2.0,0.0,'Less than 5 hours',Healthy,B.Ed,No,0.0,3.0,No,1


In [26]:
test_cleaned = test.drop(['id','City','Age','CGPA'], axis=1)
test_cleaned = pd.get_dummies(
    test_cleaned,
    columns=['Gender','Profession','Sleep Duration','Dietary Habits','Degree','Have you ever had suicidal thoughts ?','Family History of Mental Illness','Financial Stress'],
    prefix=['Gender','Profession','Sleep Duration','Dietary Habits','Degree','Have you ever had suicidal thoughts ?','Family History of Mental Illness','Financial Stress'],
    drop_first=True, dtype=int  
)
test_cleaned

,Academic Pressure,Work Pressure,Study Satisfaction,Job Satisfaction,Work/Study Hours,Depression,Gender_Male,Profession_'Educational Consultant',Profession_Chef,Profession_Pharmacist,...,Degree_MHM,Degree_MSc,Degree_Others,Degree_PhD,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes,Financial Stress_2.0,Financial Stress_3.0,Financial Stress_4.0,Financial Stress_5.0
0,2.0,0.0,3.0,0.0,10.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2.0,0.0,5.0,0.0,3.0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,3.0,0.0,4.0,0.0,4.0,0,1,0,0,0,...,0,0,0,0,1,1,0,1,0,0
3,5.0,0.0,2.0,0.0,8.0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5.0,0.0,2.0,0.0,0.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5576,2.0,0.0,5.0,0.0,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5577,1.0,0.0,3.0,0.0,11.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5578,4.0,0.0,3.0,0.0,10.0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
5579,1.0,0.0,4.0,0.0,11.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [27]:
test_actual = test_cleaned['Depression']
test_data = test_cleaned.drop(['Depression'],axis=1)
test_data = sm.add_constant(test_data)
test_data

,const,Academic Pressure,Work Pressure,Study Satisfaction,Job Satisfaction,Work/Study Hours,Gender_Male,Profession_'Educational Consultant',Profession_Chef,Profession_Pharmacist,...,Degree_MHM,Degree_MSc,Degree_Others,Degree_PhD,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes,Financial Stress_2.0,Financial Stress_3.0,Financial Stress_4.0,Financial Stress_5.0
0,1.0,2.0,0.0,3.0,0.0,10.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1.0,2.0,0.0,5.0,0.0,3.0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,1.0,3.0,0.0,4.0,0.0,4.0,1,0,0,0,...,0,0,0,0,1,1,0,1,0,0
3,1.0,5.0,0.0,2.0,0.0,8.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.0,5.0,0.0,2.0,0.0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5576,1.0,2.0,0.0,5.0,0.0,2.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5577,1.0,1.0,0.0,3.0,0.0,11.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5578,1.0,4.0,0.0,3.0,0.0,10.0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
5579,1.0,1.0,0.0,4.0,0.0,11.0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [28]:
#manage possible error and unmatch column data
train_cols = set(x.columns)
test_cols = set(test_data.columns)

print("\nIn train but not in test:")
print(train_cols - test_cols)

print("\nIn test but not in train:")
print(test_cols - train_cols)


In train but not in test:
{'CGPA', "Profession_'UX/UI Designer'", "Profession_'Content Writer'", 'Profession_Doctor', 'Age', 'Financial Stress_?', 'Profession_Architect', "Profession_'Digital Marketer'", 'Profession_Lawyer', 'Profession_Entrepreneur', 'Profession_Manager'}

In test but not in train:
{'Profession_Chef', "Profession_'Educational Consultant'"}


This check is a debugging step that protects model from crashing due to mismatched feature columns between training and testing datasets — especially common after categorical encoding.

In [29]:
def confusion_matrix(data,actual_values,model):
        
        # Confusion matrix 
        
        # Parameters
        # ----------
        # data: data frame or array
            # data is a data frame formatted in the same way as your input data (without the actual values)
            # e.g. const, var1, var2, etc. Order is very important!
        # actual_values: data frame or array
            # These are the actual values from the test_data
            # In the case of a logistic regression, it should be a single column with 0s and 1s
            
        # model: a LogitResults object
            # this is the variable where you have the fitted model 
            # e.g. results_log in this course
        # ----------
        
        #Predict the values using the Logit model
        pred_values = model.predict(data)
        # Specify the bins 
        bins=np.array([0,0.5,1])
        # Create a histogram, where if values are between 0 and 0.5 tell will be considered 0
        # if they are between 0.5 and 1, they will be considered 1
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # Calculate the accuracy
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # Return the confusion matrix and the accuracy
        return cm, accuracy

In [30]:
x_test_aligned = test_data.reindex(columns=x.columns, fill_value=0)

In [31]:
cm = confusion_matrix(x_test_aligned,test_actual,results_log)
cm

(array([[ 811., 1532.],
        [  40., 3198.]]),
 0.7183300483784268)

In [32]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,811.0,1532.0
Actual 1,40.0,3198.0


<ul>
  <li><b>True Negatives (TN) = 811</b>: The model correctly predicted 811 individuals as not depressed.</li>
  <li><b>False Positives (FP) = 1532</b>: These were predicted as depressed but were actually not.</li>
  <li><b>False Negatives (FN) = 40</b>: These were predicted as not depressed, but actually were.</li>
  <li><b>True Positives (TP) = 3198</b>: The model correctly predicted 3198 individuals as depressed.</li>
</ul>
<p>The overall prediction accuracy of the model on this unseen test dataset is:</p>
<pre><code><b>Accuracy = 0.7183</b> (or 71.83%)</code></pre>
<p>
The model performs reasonably well, achieving over 71% accuracy on new data. It shows strong performance in detecting actual depression cases (high TP = 3198) and a low false negative rate (FN = 40), which is critical in mental health-related models. However, the number of false positives is relatively high (FP = 1532), which may lead to over-predicting depression. This trade-off may be acceptable depending on the priority of sensitivity (catching as many real cases as possible). Fine-tuning thresholds or using other metrics (e.g. precision, recall, F1-score) may further improve the model.
</p>


In [33]:
print ('Missclassification rate: '+str((1532+40)/5581))

Missclassification rate: 0.2816699516215732


<h2>Conclusion</h2>

<p>
This project aimed to build a logistic regression model to predict the likelihood of student depression based on various academic, lifestyle, and personal attributes using a dataset containing over 27,000 observations. The data was preprocessed by cleaning, handling missing values, encoding categorical variables using one-hot encoding, and standardizing numerical features.
</p>

<p>
The training data was split using an 80:20 ratio, with 80% used to train the model and 20% reserved as a hold-out set for testing. During training, multicollinearity was assessed using the Variance Inflation Factor (VIF), and highly collinear variables such as <b>Profession_Student</b>, <b>Age</b>, and <b>CGPA</b> were removed to ensure model stability.
</p>

<p>
The logistic regression model was trained using the <code>statsmodels</code> library. During evaluation on the training data, the model achieved an accuracy of <b>~85%</b>, showing strong performance in capturing depression-related patterns in the data. 
</p>

<p>
When tested on the unseen 20% test set, the model achieved an accuracy of <b>71.83%</b>. The confusion matrix revealed that the model was especially effective in identifying students who are actually depressed (True Positives = 3198), though it did generate a fair number of false positives (1532), indicating a tendency to overpredict depression. However, the model kept false negatives (40) quite low, which is important in mental health screening.
</p>

<p>
Overall, the model generalizes reasonably well to new data and provides valuable predictive power for identifying students at risk of depression. Further improvements could involve hyperparameter tuning, feature engineering, and potentially exploring more advanced classification models to improve precision and reduce false positives.
</p>
